In [ ]:
"""
Veri seti yüklendi, sınıflar ve özelliklerin başlıkları güncellendi.
"""

import pandas as pd
df = pd.read_csv("../data/drug_consumption.data", header=None)
new_columns = [
    "ID", "Age", "Gender", "Education", "Country", "Ethnicity",
    "Nscore", "Escore", "Oscore", "Ascore", "Cscore",
    "Impulsiveness", "SS", "Alcohol", "Amphetamines", "Amyl_nitrite",
    "Benzodiazepines", "Cannabis", "Chocolate", "Cocaine", "Caffeine",
    "Crack", "Ecstasy", "Heroin", "Ketamine", "Legal_highs", "LSD",
    "Methadone", "Mushrooms", "Nicotine", "VSA", "Semeron"
]

df.columns = new_columns

In [ ]:
"""
Madde kullanım süreleri, numerik hale getirildi.

0 (Veri Setinde: CL0): Hiç Kullanmamış
1 (Veri Setinde: CL1): 10+ Yıl Önce Kullanmış
2 (Veri Setinde: CL2): Son 10 Yıl İçinde Kullanmış
3 (Veri Setinde: CL3): Son 1 Yıl İçinde Kullanmış
4 (Veri Setinde: CL4): Son 1 Ay İçinde Kullanmış
5 (Veri Setinde: CL5): Son 1 Hafta İçinde Kullanmış
6 (Veri Setinde: CL6): Son 24 Saat İçinde Kullanmış

"""

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in df.columns:
    if df[col].dtype == 'object' and col not in ["ID", "Cocaine_binary"]:
        df[col] = le.fit_transform(df[col])


In [ ]:
"""

Kokain, araştırmaya konu olan uyuşturucu olarak seçildi.
Kokain Kullanım Durumunu gösteren bir sınıf oluşturuldu (binary class).
(0,1): Kullanmıyor - 0,
[2,6]: Kullanıyor - 1.

"""

df["Cocaine_binary"] = df["Cocaine"].apply(
    lambda x: 1 if x >= 2 else 0
)

In [ ]:
"""

Özellik seçimi ve modellerin eğitimi için gereken train-test ayrımı yapıldı.
X: Bağımsız Değişken(ler)
Y: Bağımlı Değişken 

"""

from sklearn.model_selection import train_test_split

X = df.drop("Cocaine_binary", axis=1)
y = df["Cocaine_binary"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [ ]:
"""

Feature Selection 1: Information Gain ile En Değerli Özellikleri Belirleme

Entropi ve Information Gain hesapları yapıldı.
En yüksek Bilgi Kazancını veren 15 Özellik listelendi.


"""

from sklearn.feature_selection import mutual_info_classif

mi_scores = mutual_info_classif(X, y, random_state=42)
mi_df = pd.DataFrame({
    "feature": X.columns,
    "mi_score": mi_scores
}).sort_values(by="mi_score", ascending=False)

mi_df.head(15)

In [ ]:
"""

Feature Selection 2: ReliefF ile En İyi Ayrım Yaptırmaya Yarayan Özellikleri Belirleme


"""

from skrebate import ReliefF

relief = ReliefF(n_neighbors=100)
relief.fit(X_train.values, y_train.values)

relief_scores = relief.feature_importances_

relief_df = pd.DataFrame({
    "feature": X.columns,
    "relief_score": relief_scores
}).sort_values(by="relief_score", ascending=False)

relief_df.head(15)

In [ ]:
""" 
Min-Max Normalizasyonu ile değerleri 0-1 arasına çekelim: Ki-Kare Testi için gerekli.
"""

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
"""

Feature Selection 3: Ki-Kare Testi.

"""

from sklearn.feature_selection import SelectKBest, chi2

chi2_selector = SelectKBest(score_func=chi2, k='all')
chi2_selector.fit(X_scaled, y)
chi2_scores = chi2_selector.scores_

In [ ]:
"""

Feature Selection 3: Ki-Kare Testi.
Feature'lar ile hedef değişken arasındaki ilişkilerin gösterimi gerçekleştiriliyor.
En güçlü ilişkiye sahip olan 15 feature listelenmiştir.

"""

chi2_df = pd.DataFrame({
    "feature": X.columns,
    "chi2_score": chi2_scores
}).sort_values(by="chi2_score", ascending=False)

chi2_df.head(15)